In [1]:
#default_exp hierarchical

In [2]:
#hide
%load_ext autoreload
%autoreload 2

# Hierarchical Time Series Forecasting

In [3]:
#hide
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [4]:
#hide
from fastcore.test import test_eq

In [5]:
#export
from typing import Callable, List, Optional, Tuple

import numpy as np
import pandas as pd

from statsforecast.core import StatsForecast

In [6]:
#export
def _aggregate_key(df: pd.DataFrame, keys: List[List[str]], agg_fn: Callable = np.sum):
    """Aggregates `df` according to `keys` using `agg_fn`."""
    df = df.copy()
    max_len_idx = np.argmax([len(key) for key in keys])
    bottom_comb = keys[max_len_idx]
    orig_cols = df.drop(labels=['ds', 'y'], axis=1).columns.to_list()
    df_keys = []
    for key in keys:
        if key == ['total']:
            df = df.assign(total='total')
        df_key = df.groupby(key + ['ds'])['y'].apply(agg_fn).reset_index()
        df_key['unique_id'] = df_key[key].agg('_'.join, axis=1)
        if key == bottom_comb:
            bottom_keys = df_key['unique_id'].unique()
        df_keys.append(df_key)
    df_keys = pd.concat(df_keys)
    s_df = df_keys[['unique_id'] + orig_cols].drop_duplicates().reset_index(drop=True)
    s_df = s_df.set_index('unique_id')
    y_df = df_keys[['unique_id', 'ds', 'y']].set_index('unique_id')
    #s_mat definition
    s_mat = np.zeros((len(s_df), len(bottom_keys)))
    for idx, label in enumerate(s_df.index, start=0):
        if label == 'total':
            s_mat[idx] = 1
        else:
            s_mat[idx, [label in bt for bt in bottom_keys]] = 1
    return s_df, s_mat, y_df

In [7]:
#hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
s_df, s_mat, y_df = _aggregate_key(df, [['total'],
                                         ['State'], 
                                         ['Purpose'], 
                                         ['State', 'Region'], 
                                         ['State', 'Purpose'], 
                                         ['State', 'Region', 'Purpose']])
test_eq(len(y_df), 34_000)
test_eq(y_df.index.nunique(), 425)
test_eq(s_mat.shape, (425, 304))

In [8]:
#export
def bottom_up(y_hat: np.ndarray, s_mat: np.ndarray):
    # size of y_hat = n_levels, horizon
    _, h = y_hat.shape
    n_levels, n_bottom = s_mat.shape
    g_mat = np.eye(n_bottom, n_levels, k=(n_levels - n_bottom))
    s_g_mat = s_mat @ g_mat
    return np.vstack([s_g_mat @ y_hat[:,i] for i in range(h)]).T

In [16]:
#export
class HierarchicalStatsForecast:
    
    def __init__(self, df: pd.DataFrame, keys: List[List[str]], 
                 models: List, freq: str, 
                 n_jobs: int = 1, ray_address: Optional[str] = None):
        self.s_df, self.s_mat, y_df = _aggregate_key(df, keys=keys)
        self.fcst = StatsForecast(df=y_df, models=models, freq=freq, 
                                  n_jobs=n_jobs, ray_address=ray_address,
                                  sort_df=False)
    
    def forecast(self, h: int, reconcile_fns: List[Callable] = [bottom_up],
                 xreg: Optional[pd.DataFrame] = None, 
                 level: Optional[Tuple] = None):
        fcsts = self.fcst.forecast(h=h, xreg=xreg, level=level)
        model_names = fcsts.drop(columns=['ds'], axis=1).columns.to_list()
        for model_name in model_names:
            fcsts_model = fcsts[model_name].values.reshape(-1, h)
            for reconcile_fn in reconcile_fns:
                reconcile_fn_name = reconcile_fn.__name__
                fcsts_model = reconcile_fn(fcsts_model, self.s_mat)
                fcsts[f'{reconcile_fn_name}_{model_name}'] = fcsts_model.flatten()
        return fcsts

In [10]:
from statsforecast.models import naive

In [11]:
ds_int = df[['ds']].drop_duplicates().assign(ds_int = lambda df: np.arange(len(df)) + 1)

In [12]:
df = df.merge(ds_int, how='left', on=['ds']).drop('ds', axis=1)

In [13]:
df = df.rename(columns={'ds_int': 'ds'})

In [17]:
hier_fcst = HierarchicalStatsForecast(df, 
                                      keys=[['total'],
                                            ['State'], 
                                            ['Purpose'], 
                                            ['State', 'Region'], 
                                            ['State', 'Purpose'], 
                                            ['State', 'Region', 'Purpose']],
                                      models=[naive],
                                      freq='D', 
                                      n_jobs=-1)

In [18]:
hier_fcst.forecast(10)

,ds,naive,bottom_up_naive
unique_id,,,
total,81,27593.554688,27593.554198
total,82,27593.554688,27593.554198
total,83,27593.554688,27593.554198
total,84,27593.554688,27593.554198
total,85,27593.554688,27593.554198
...,...,...,...
Western Australia_Experience Perth_Visiting,86,455.316711,455.316711
Western Australia_Experience Perth_Visiting,87,455.316711,455.316711
Western Australia_Experience Perth_Visiting,88,455.316711,455.316711
